# ===========================================
# 5_rag_generation.ipynb
# Kombination von Retrieval (BM25) und Transformer-Generierung (RAG)
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm

# Sicherstellen, dass NLTK-Daten vorhanden sind
nltk.download('punkt')
%matplotlib inline

# Zelle 2: Dummy-Wissensbasis laden (in der Praxis aus einer CSV)
# Hier simulieren wir eine kleine Wissensbasis als Liste von Textpassagen.
knowledge_documents = [
    "Neuronale Netze sind Algorithmen, die lose vom menschlichen Gehirn inspiriert sind und zur Mustererkennung eingesetzt werden.",
    "Machine Learning ist ein Teilbereich der künstlichen Intelligenz, der es Computern ermöglicht, aus Daten zu lernen.",
    "Deep Learning verwendet tiefe neuronale Netzwerke, um komplexe Muster und Strukturen in großen Datensätzen zu erkennen.",
    "Die Relativitätstheorie wurde von Albert Einstein entwickelt und beschreibt die Gravitation als Folge der Krümmung von Raum und Zeit.",
    "Künstliche Intelligenz wird in vielen Bereichen eingesetzt, von der Medizin bis zur autonomen Fahrzeugsteuerung."
]

# Zelle 3: BM25-Retriever aufbauen
# Tokenisiere alle Dokumente
tokenized_docs = [word_tokenize(doc.lower()) for doc in knowledge_documents]
bm25 = BM25Okapi(tokenized_docs)

# Zelle 4: Eingabe-Frage und Retrieval
input_question = "Wie funktionieren neuronale Netze?"
tokenized_question = word_tokenize(input_question.lower())

# Abrufen der Top 3 relevanten Dokumente
top_k = 3
retrieved_indices = bm25.get_top_n(tokenized_question, knowledge_documents, n=top_k)
print("Top abgerufene Dokumente:")
for doc in retrieved_indices:
    print("- ", doc)

# Zelle 5: Prompt für den Transformer generieren
# Kombiniere die abgerufenen Dokumente in einem Prompt
prompt = "Frage: " + input_question + "\nWissensbasis:\n"
for idx, doc in enumerate(retrieved_indices, 1):
    prompt += f"{idx}. {doc}\n"
prompt += "Antwort:"

print("\nGenerierungsprompt:\n", prompt)

# Zelle 6: Transformer (T5) zur Antwortgenerierung
device = "cuda" if torch.cuda.is_available() else "cpu"
t5_model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)
t5_model.to(device)
t5_model.eval()

# Tokenisiere den Prompt
inputs = t5_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generiere Antwort mit Beam Search
output_ids = t5_model.generate(
    **inputs,
    num_beams=4,
    max_length=150,
    early_stopping=True
)
generated_answer = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\nGenerierte Antwort:")
print(generated_answer)

# Zelle 7: Fazit & Ausblick
print("""
Fazit:
- Der BM25-Retriever extrahiert relevante Textpassagen aus der Wissensbasis basierend auf der Eingabefrage.
- Die abgerufenen Dokumente werden in einen Prompt integriert, der an ein vortrainiertes T5-Modell übergeben wird.
- Das T5-Modell generiert auf Basis dieses kombinierten Prompts eine faktenbasierte Antwort.
Nächste Schritte:
- Integration einer größeren Wissensdatenbank (z. B. mit FAISS für schnelle Vektor-Suche).
- Feinabstimmung des Transformer-Modells (z. B. mit RLHF), um die Antwortqualität weiter zu verbessern.
- Evaluation der Antworten mittels geeigneter Metriken.
""")
